In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, sin, cos, asin, sqrt, pi, atan, atan2, fabs

In [2]:
def calcular_distancia(lat1,long1,lat2,long2):
    lon1, lat1 = (radians(coord) for coord in (long1,lat1))
    lon2, lat2 = (radians(coord) for coord in (long2,lat2))
    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    a = (
        sin(dlat * 0.5)**2 +
        cos(lat1) * cos(lat2) * sin(dlon * 0.5)**2
    )
    
    radioTierra = 6371008.8
    return 2 * radioTierra * asin(sqrt(a)) / 1000

#distancia de Haversine
#https://gist.github.com/habibutsu/8bbcc202a915e965c6a6d4f561d0e482

In [5]:
def distancia_minima(fila, dataframe):
    lat, lon = fila
    distancias = []
    for i in range (len(dataframe)):
        d = calcular_distancia(lat,lon,dataframe.lat[i], dataframe.lon[i])
        distancias.append(d)
    return min(distancias)

In [6]:
def cantidad_a_distancia(fila, dist, dataframe):
    lat, lon = fila
    cantidad = 0
    for i in range (len(dataframe)):
        d = calcular_distancia(lat,lon,dataframe.lat[i], dataframe.lon[i])
        if d <= dist:
            cantidad += 1
    return cantidad

## Propiedades solo de Capital

In [ ]:
propiedades = pd.read_csv('properati.csv')
#propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/properati.csv')

In [ ]:
solo_capital = propiedades.loc[(propiedades.state_name == 'Capital Federal') & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),:]
solo_capital = solo_capital.loc[((solo_capital.lat <= -34) & (solo_capital.lat >= -35)),:]
solo_capital = solo_capital.loc[((solo_capital.lon <= -58) & (solo_capital.lon >= -59)),:]

## Subtes

In [ ]:
subtes = pd.read_csv('datos/estaciones-de-subte.csv', sep = ",")

In [ ]:
subtes.rename(columns = {'X': 'lon', 'Y': 'lat'}, inplace = True)

In [ ]:
solo_capital.loc[:,'distancia_subtes'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, subtes), axis = 1)

## Monumentos

In [7]:
monumentos = pd.read_csv('datos/monumentos.csv', sep = ",")

In [9]:
solo_capital.loc[:,'distancia_monumentos'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, monumentos), axis = 1)

## Ferrocarriles

In [13]:
ferrocarriles = pd.read_csv('datos/estaciones-de-ferrocarril.csv', sep = ";")

In [14]:
ferrocarriles.rename(columns = {'LAT': 'lat', 'LNG': 'lon' }, inplace = True)

In [ ]:
solo_capital.loc[:,'distancia_ferrocarril'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, ferrocarriles), axis = 1)

## Estadios

In [21]:
estadios = pd.read_csv('datos/estadios.csv', sep = ";")

In [20]:
estadios.rename(columns = {'LAT': 'lat', 'LONG': 'lon' }, inplace = True)

In [35]:
solo_capital.loc[:,'distancia_estadios'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, estadios), axis = 1)

NameError: name 'solo_capital' is not defined

## Colectivos

In [22]:
colectivos = pd.read_csv('datos/paradas-de-colectivo.csv', sep = ";")
colectivos.head()

,X,Y,STOP_ID,TIPO,CALLE,NUMERO,ENTRE1,ENTRE2,LINEAS,DIR_NORM,CALLE_NORM,ALTURA_NOR,COORDX,COORDY,METROBUS,STOP_NAME,STOP_DESC,FUENTE,VERIFICADA,FECHA_ULTI
0,"-58,4327908137487","-34,5923809470",1000086,PARADAS CON REFUGIO,CORDOBA AV.,4859,ARMENIA,GURRUCHAGA,"{151,168}",CORDOBA AV. 4859,CORDOBA AV.,4859.0,102.798.829.931.999.000.000,104.091.825.983.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,9/4/2015
1,"-58,5140298647844","-34,6016371880",1005465,PARADA NO OFICIAL,NUEVA YORK,4224,NaN,GUALEGUAYCHU,{105},NUEVA YORK 4224,NUEVA YORK,4224.0,95.346.600.539.799.900.000,103.064.210.534.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,17/4/2015
2,"-58,4497660921390","-34,5544664433",1002096,PARADAS CON REFUGIO,DEL LIBERTADOR AV.,6250,BLANCO ENCALADA,OLAZABAL,"{15,29}",DEL LIBERTADOR AV. 6250,DEL LIBERTADOR AV.,6250.0,101.242.081.519.000.000.000,108.298.117.206.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,10/4/2015
3,"-58,4941044948861","-34,5623407465",1006607,PARADAS CON REFUGIO,"LARRALDE, CRISOLOGO AV.",5187,NaN,NaN,"{110,71}","LARRALDE, CRISOLOGO AV. 5187","LARRALDE, CRISOLOGO AV.",5187.0,97.172.965.995.399.900.000,107.424.229.065.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,14/4/2015
4,"-58,4844741311517","-34,5757583663",1004767,PARADAS DE POSTE,OLAZABAL AV.,4920,NaN,TRIUNVIRATO AV.,{90},OLAZABAL AV. 4920,OLAZABAL AV.,4920.0,98.057.080.321.899.900.000,105.936.009.144.000.000.000,F,NaN,NaN,RELEVAMIENTO2015,T,14/4/2015


In [36]:
estadios.rename(columns = {'X': 'lat', 'Y': 'lon' }, inplace = True)

In [37]:
def abc (fila):
    a,b = fila
    a = a.split(',')
    a = '.'.join(a)
    b = b.split(',')
    b = '.'.join(b)
    return float(a), float(b)

colectivos.loc[:,['lat', 'lon']] = colectivos.loc[:,['lat','lon']].apply(lambda x: abc(x), axis = 1)
colectivos.head()

KeyError: "None of [['lat', 'lon']] are in the [columns]"